In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("uniq_artworks_htmlLists.csv")

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.shape

(32882, 29)

In [4]:
df.head()

,_id,acquisition_year,acquisition_mode,date_creation,type,domain,domain_leaf,domain_deno_for_grid,domain_description_mst,collection,...,recap_description,all_realized_operations_history,hanging_history,hanging_history_m34,temporary_exhibitions_m30,exposition_out_of_folder,expositions,expositions_without_current,inscriptions,authors
0,150000000060949,2009.0,Don,1946,individual,Objet/Design,Presse agrumes,Objet/Design,"Presse agrumes|Aluminium, acier chromé et laqu...",NaN,...,NaN,<ul><li>2010/07/27 - 210I - Déplacement vers l...,NaN,NaN,NaN,NaN,NaN,NaN,,150000000000887
1,150000000078314,2002.0,Don,1968 - 1975,separable,Architecture,Dessin d'architecture,Architecture,Ensemble de dessins à inventorier,NaN,...,Dessins|Projet réalisé|1968-1975,<ul><li>2003/10/14 - 213I - Décrochage avec dé...,<ul><li>2003/07/16 - 1N4S34CO : Cimaise Ouest ...,<ul><li>Présentation des collections permanent...,NaN,NaN,NaN,<ul><li>Présentation des collections permanent...,,9000000000082144
2,150000000020652,1984.0,Achat,1980,individual,Photographie,Photographie,Photographie,Epreuve gélatino-argentique,NaN,...,NaN,<ul><li>2014/07/15 - 730I - Localisation inter...,<ul><li>1986/04/30 - CE+3S : 3ème Sud|Opératio...,NaN,NaN,NaN,"<ul><li>Masques : Arles (France), Les rencontr...","<ul><li>Masques : Arles (France), Les rencontr...",S.D.AU R. : Joel Peter Witkin/Arms broken by ....,9000000000067959
3,150000000077148,2001.0,Don,vers novembre 1966,separable,Architecture,Dessin d'architecture,Architecture,Perspective|Mine de plomb et crayon jaune sur ...,NaN,...,Projet réalisé|1963-1984,<ul><li>2008/09/23 - 210I - Déplacement vers l...,NaN,NaN,NaN,NaN,NaN,NaN,,9000000000081963
4,150000000476688,2011.0,Don,mars 1994 - juillet 1994,separable,Architecture,Plan,Architecture,"Dessin d'architecture, Plan|Encre de Chine, gr...",NaN,...,Projet réalisé|1992-1995,<ul><li>2011/05/10 - 210I - Déplacement vers l...,NaN,NaN,NaN,NaN,NaN,NaN,,150000000476606


# All data field

In [5]:
list(df)

['_id',
 'acquisition_year',
 'acquisition_mode',
 'date_creation',
 'type',
 'domain',
 'domain_leaf',
 'domain_deno_for_grid',
 'domain_description_mst',
 'collection',
 'collection_department',
 'recap_authors',
 'authors_list',
 'authors_nationality',
 'authors_birth_death',
 'authors_name_complement',
 'title_notice',
 'title_list',
 'comments',
 'recap_description',
 'all_realized_operations_history',
 'hanging_history',
 'hanging_history_m34',
 'temporary_exhibitions_m30',
 'exposition_out_of_folder',
 'expositions',
 'expositions_without_current',
 'inscriptions',
 'authors']

## remove html tags

In [6]:
import re

def get_list_from_html(field):
    if(pd.isnull(field)):
        return []
    else:
        regex_item = re.compile('<li>(.*?)<\/li>', flags=re.S)
        tab = regex_item.split(str(field))
        new_tab = []
        for item in tab:
            if len(item) > 0 and 'ul>' not in item and item != '\n':
                new_tab.append(item)
        return new_tab

In [7]:
df["operation_list"] = df["all_realized_operations_history"].apply(get_list_from_html)

## remove no code cases from operation list

In [8]:
get_cleaned_list = lambda x : [i for i in x if len(i.split('-')) > 2]
remove_invalid_date = lambda x : [i for i in x if (int(i.split('-')[0][:4])>1678 and int(i.split('-')[0][:4])<2018)]
df["operation_list_clean"] = df["operation_list"].apply(get_cleaned_list).apply(remove_invalid_date)

In [9]:
# get_operation_between= lambda x : [i for i in x if (int(i.split('-')[0][:4])>1997 and int(i.split('-')[0][:4])<2001)]
# df["operation_list_1997_2000"] = df["operation_list_clean"].apply(get_operation_between)

## operation after 2001

In [10]:
# get_operation_after = lambda x : [i for i in x if (int(i.split('-')[0][:4])>2000)]
# df["operation_list_after_2001"] = df["operation_list_clean"].apply(get_operation_after)

## count operation times

In [11]:
df["operation_times"] = df["operation_list_clean"].apply(lambda x: len(x))

In [12]:
df_freq = df[["_id", "title_notice","domain", "authors_list", "date_creation", "operation_list_clean", "operation_times"]].sort_values(["operation_times"],ascending=False)

## get opt field(date, code, opt_name, details), store into dict

In [13]:
def format_date(date):
    return date.replace("/00", "/01")

In [22]:
get_opt_field = lambda x : sorted([{"date": format_date(i.split(' - ')[0]), "opt_code":i.split(" - ")[1], "opt_name":i.split(" - ")[2]} if len(i.split(" - "))<4 else {"date": format_date(i.split(' - ')[0]), "opt_code":i.split(" - ")[1], "opt_name": i.split(" - ")[2], "opt_detail": '-'.join(i.split(" - ")[3:])} for i in x], key=lambda d: d['date'])

In [23]:
df_freq["opt_field"] = df_freq["operation_list_clean"].apply(get_opt_field)

In [25]:
df_freq.shape

(32882, 8)

In [26]:
# opt_code_list = lambda x : [i.split(' - ')[1] for i in x]
def opt_code_freq(x):
    l = [i.split(' - ')[1] for i in x]
    #return  dict((x,l.count(x)) for x in set(l))
    opt_count_l = [{"code": x, "count": l.count(x)} for x in set(l)]
    return sorted(opt_count_l, key=lambda d: d['count'], reverse=True)

In [27]:
df_freq["opt_freq"] = df_freq["operation_list_clean"].apply(opt_code_freq)

In [28]:
df_freq["opt_diversity"] = df_freq["opt_freq"].apply(lambda x:len(x))

In [29]:
df_freq.sort_values("opt_diversity", ascending = False)

,_id,title_notice,domain,authors_list,date_creation,operation_list_clean,operation_times,opt_field,opt_freq,opt_diversity
11985,150000000007685,Big Electric Chair|(Grande chaise électrique),Peinture,"WARHOL Andy (WARHOLA Andrew, dit)",décembre 1967 - janvier 1968,[2017/10/09 - 230E - Sortie vers loc. externe ...,145,"[{'date': '1977/03/15', 'opt_code': '211I', 'o...","[{'code': '211I', 'count': 32}, {'code': '210I...",22
7349,150000000011460,Sphinx,Peinture,"PICABIA Francis (MARTINEZ DE PICABIA Francis, ...",[1929],[2017/04/13 - 210I - Déplacement vers loc. int...,156,"[{'date': '1970/10/12', 'opt_code': '211I', 'o...","[{'code': '211I', 'count': 36}, {'code': '210I...",22
2930,150000000017244,Homage to the Square|(Hommage au carré),Peinture,ALBERS Josef,1958,[2014/07/06 - 242E - Retour à un lieu de dépôt...,43,"[{'date': '1986/01/01', 'opt_code': '299I', 'o...","[{'code': '210I', 'count': 9}, {'code': '410',...",21
5920,150000000027848,La liseuse,Peinture,PICASSO Pablo,1920,[2017/10/06 - 212I - Accrochage avec déplaceme...,153,"[{'date': '1970/09/16', 'opt_code': '211I', 'o...","[{'code': '210I', 'count': 36}, {'code': '211I...",21
24851,150000000023294,Auf Weiss II|(Sur blanc II),Peinture,KANDINSKY Vassily,1923,[2016/09/01 - 210I - Déplacement vers loc. int...,141,"[{'date': '1985/11/18', 'opt_code': '299I', 'o...","[{'code': '210I', 'count': 37}, {'code': '410'...",21
8387,150000000014769,La Rue pavoisée,Peinture,DUFY Raoul,1906,[2017/09/04 - 212I - Accrochage avec déplaceme...,149,"[{'date': '1980/01/01', 'opt_code': '410', 'op...","[{'code': '210I', 'count': 34}, {'code': '211I...",21
27907,150000000013380,Les chevaux morts,Peinture,MASSON André,1927,[2017/01/06 - 210I - Déplacement vers loc. int...,106,"[{'date': '1986/01/01', 'opt_code': '299I', 'o...","[{'code': '210I', 'count': 33}, {'code': '211I...",21
23411,150000000011502,Composition A.XX,Peinture,MOHOLY-NAGY Laszlo,1924,[2016/09/26 - 210I - Déplacement vers loc. int...,117,"[{'date': '1970/09/10', 'opt_code': '211I', 'o...","[{'code': '211I', 'count': 29}, {'code': '210I...",20
25484,150000000015569,Nu dans la nature,Peinture,"LAM Wifredo (LAM Y CASTILLA Wifredo Oscar, dit)",[1944],[2017/01/17 - 221I - Retour vers loc. interne ...,90,"[{'date': '1985/12/02', 'opt_code': '299I', 'o...","[{'code': '211I', 'count': 18}, {'code': '210I...",20
11565,150000000014966,Chinon n° 645,Peinture,RICHTER Gerhard,1987,[2017/06/22 - 212I - Accrochage avec déplaceme...,87,"[{'date': '1988/05/03', 'opt_code': '211I', 'o...","[{'code': '210I', 'count': 18}, {'code': '410'...",20


In [30]:
df_freq.to_csv("artwork_operations.csv", index=False)

In [263]:
df_freq

,_id,title_notice,domain,authors_list,date_creation,operation_list_clean,operation_times,opt_field,opt_freq,opt_diversity
28945,150000000030351,Nature morte à la table de marbre vert,Peinture,MATISSE Henri,septembre 1941,[2016/10/03 - 221I - Retour vers loc. interne ...,173,"[{'date': '2016/10/03', 'opt_code': '221I', 'o...","[{'code': '210I', 'count': 52}, {'code': '211I...",16
21275,150000000017627,Le Labyrinthe,Peinture,MASSON André,1938,[2017/01/06 - 210I - Déplacement vers loc. int...,162,"[{'date': '2017/01/06', 'opt_code': '210I', 'o...","[{'code': '210I', 'count': 54}, {'code': '410'...",18
21308,150000000022395,Nature morte au magnolia,Peinture,MATISSE Henri,octobre 1941,[2017/10/20 - 230E - Sortie vers loc. externe ...,160,"[{'date': '2017/10/20', 'opt_code': '230E', 'o...","[{'code': '210I', 'count': 44}, {'code': '410'...",16
21695,150000000026052,Portrait de Dédie,Peinture,MODIGLIANI Amedeo,[1918],[2017/10/06 - 212I - Accrochage avec déplaceme...,158,"[{'date': '2017/10/06', 'opt_code': '212I', 'o...","[{'code': '210I', 'count': 39}, {'code': '211I...",18
21670,150000000015948,"Painting (Silver over Black, White, Yellow and...",Peinture,POLLOCK Jackson,1948,[2017/10/10 - 230E - Sortie vers loc. externe ...,157,"[{'date': '2017/10/10', 'opt_code': '230E', 'o...","[{'code': '211I', 'count': 53}, {'code': '210I...",17
7349,150000000011460,Sphinx,Peinture,"PICABIA Francis (MARTINEZ DE PICABIA Francis, ...",[1929],[2017/04/13 - 210I - Déplacement vers loc. int...,156,"[{'date': '2017/04/13', 'opt_code': '210I', 'o...","[{'code': '211I', 'count': 36}, {'code': '210I...",22
19851,150000000015884,Le faubourg de Collioure,Peinture,DERAIN André,[1905],[2017/09/18 - 210I - Déplacement vers loc. int...,153,"[{'date': '2017/09/18', 'opt_code': '210I', 'o...","[{'code': '210I', 'count': 35}, {'code': '211I...",18
5920,150000000027848,La liseuse,Peinture,PICASSO Pablo,1920,[2017/10/06 - 212I - Accrochage avec déplaceme...,153,"[{'date': '2017/10/06', 'opt_code': '212I', 'o...","[{'code': '210I', 'count': 36}, {'code': '211I...",21
1541,150000000023616,Portrait du sculpteur Oscar Miestchaninoff,Peinture,SOUTINE Chaïm,[1923 - 1924],[2017/10/10 - 230E - Sortie vers loc. externe ...,150,"[{'date': '2017/10/10', 'opt_code': '230E', 'o...","[{'code': '211I', 'count': 37}, {'code': '210I...",17
8387,150000000014769,La Rue pavoisée,Peinture,DUFY Raoul,1906,[2017/09/04 - 212I - Accrochage avec déplaceme...,149,"[{'date': '2017/09/04', 'opt_code': '212I', 'o...","[{'code': '210I', 'count': 34}, {'code': '211I...",21


In [32]:
df_freq.iloc[0].to_json("example_artwork.json")

## export to json

In [31]:
df_freq.to_json("artwork_operations.json", orient="records")

# Operation Counts

In [265]:
optCodes = lambda x : [ {"date": format_date(item.split(" - ")[0]), "code":item.split(" - ")[1], "opt_name":item.split(" - ")[2]} for item in x]

In [266]:
all_opts = list(df_freq["operation_list_clean"].apply(optCodes))

In [267]:
all_opts_joined = [i for row in all_opts for i in row]

In [268]:
all_opts_joined

[{'code': '221I',
  'date': '2016/10/03',
  'opt_name': 'Retour vers loc. interne'},
 {'code': '230E',
  'date': '2016/06/01',
  'opt_name': 'Sortie vers loc. externe'},
 {'code': '221I',
  'date': '2016/05/20',
  'opt_name': 'Retour vers loc. interne'},
 {'code': '410', 'date': '2016/04/01', 'opt_name': 'Infos habillage'},
 {'code': '240E',
  'date': '2015/11/27',
  'opt_name': "Transport entre deux lieux d'exposition"},
 {'code': '230E',
  'date': '2015/06/15',
  'opt_name': 'Sortie vers loc. externe'},
 {'code': '302', 'date': '2015/05/22', 'opt_name': 'Constat départ'},
 {'code': '210I',
  'date': '2015/05/20',
  'opt_name': 'Déplacement vers loc. interne'},
 {'code': '322', 'date': '2015/03/15', 'opt_name': 'Restauration'},
 {'code': '210I',
  'date': '2015/02/26',
  'opt_name': 'Déplacement vers loc. interne'},
 {'code': '210I',
  'date': '2015/02/25',
  'opt_name': 'Déplacement vers loc. interne'},
 {'code': '210I',
  'date': '2014/09/22',
  'opt_name': 'Déplacement vers loc. in

In [269]:
df_opts = pd.DataFrame(data=all_opts_joined)

In [270]:
df_opts

,code,date,opt_name
0,221I,2016/10/03,Retour vers loc. interne
1,230E,2016/06/01,Sortie vers loc. externe
2,221I,2016/05/20,Retour vers loc. interne
3,410,2016/04/01,Infos habillage
4,240E,2015/11/27,Transport entre deux lieux d'exposition
5,230E,2015/06/15,Sortie vers loc. externe
6,302,2015/05/22,Constat départ
7,210I,2015/05/20,Déplacement vers loc. interne
8,322,2015/03/15,Restauration
9,210I,2015/02/26,Déplacement vers loc. interne


In [271]:
def find_wrong_date(date):
    if("/00" in date):
        print(date)

In [272]:
pd.Timestamp.min

Timestamp('1677-09-21 00:12:43.145225')

In [274]:
# df_opts["date"].apply(find_wrong_date)

In [275]:
df_opts['date'] = pd.to_datetime(df_opts["date"])

In [276]:
df_opts.sort_values("date", ascending=False)

,code,date,opt_name
324662,210I,2017-12-12,Déplacement vers loc. interne
305770,322,2017-11-29,Restauration
311646,322,2017-11-15,Restauration
314478,301,2017-11-13,Constat entrée
112452,302,2017-11-01,Constat départ
29886,302,2017-11-01,Constat départ
85136,302,2017-11-01,Constat départ
43061,302,2017-10-26,Constat départ
306358,322,2017-10-25,Restauration
133848,302,2017-10-24,Constat départ


In [191]:
df_opts_rank = df_opts.groupby(["code", "opt_name"]).size().reset_index(name='count')

In [192]:
df_opts_rank = df_opts_rank.sort_values("count", ascending=False) 

In [193]:
df_opts_rank

,code,opt_name,count
3,210I,Déplacement vers loc. interne,81415
5,211I,Saisie localisation interne,24065
15,230E,Sortie vers loc. externe,14013
14,221I,Retour vers loc. interne,13755
29,302,Constat départ,13056
6,212I,Accrochage avec déplacement vers loc. interne,10729
37,410,Infos habillage,8971
56,901,Validation inventaire réglementaire,7398
8,213I,Décrochage avec déplacement vers loc. interne,7113
33,306,Constat,6307


In [64]:
df_opt_rank.to_csv("opt_rank.csv", index=False)

In [218]:
df_opts.to_csv("all_operations.csv", index=False)